In [1]:
import math
import feedparser
import requests
from bs4 import BeautifulSoup
import spacy
from collections import Counter
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from datetime import datetime, timedelta
import time
import logging
import backoff
from transformers import BertTokenizer, BertForTokenClassification, pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import pipeline as transformers_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import io
import os
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap, MarkerCluster
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import schedule
import json
import hdbscan
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
import tweepy

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Download necessary NLTK data
try:
    nltk.download('vader_lexicon', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    logger.warning(f"Failed to download NLTK data: {e}. Sentiment analysis may not work properly.")

# Suppress warnings
warnings.filterwarnings('ignore')

c:\Users\Aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (c:\Users\Aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\__init__.py)

In [ ]:
# Define search queries for disaster types
disaster_queries = [
    # Natural Disasters
    "earthquake",
    "flood",
    "hurricane",
    "tornado",
    "tsunami",
    "wildfire",
    "drought",
    "landslide",
    "volcanic eruption",
    "extreme weather",
    # Non-Natural Disasters
    "industrial accident",
    "chemical spill",
    "radiation leak",
    "explosion",
    "airplane crash",
    "train derailment",
    "shipwreck",
    "road accident",
    "structural collapse",
    "mine accident",
    "oil spill",
    "war",
    "civil war",
    "ethnic conflict",
    "terrorist attack",
    "bombing",
    "hostage situation",
    "cyber attack",
    "urban fire",
    "industrial fire"
]

# Load the spaCy model
try:
    nlp_spacy = spacy.load('en_core_web_sm')
    logger.info("Loaded spaCy model")
except Exception as e:
    logger.error(f"Failed to load spaCy model: {e}")
    logger.info("Installing spaCy model...")
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp_spacy = spacy.load('en_core_web_sm')

# Load pre-trained BERT tokenizer and model for NER
try:
    tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
    model = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
    nlp_ner = pipeline('ner', model=model, tokenizer=tokenizer)
    logger.info("Loaded BERT NER model")
except Exception as e:
    logger.error(f"Failed to load BERT NER model: {e}")
    nlp_ner = None

# Load more advanced NER model
try:
    advanced_ner_tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
    advanced_ner_model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
    advanced_nlp_ner = transformers_pipeline('ner', model=advanced_ner_model, tokenizer=advanced_ner_tokenizer, aggregation_strategy="simple")
    logger.info("Loaded advanced NER model")
except Exception as e:
    logger.error(f"Failed to load advanced NER model: {e}")
    advanced_nlp_ner = None

# Load disaster classification model
try:
    disaster_classifier_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
    disaster_classifier_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
    disaster_classifier = transformers_pipeline("zero-shot-classification", model=disaster_classifier_model, tokenizer=disaster_classifier_tokenizer)
    logger.info("Loaded disaster classification model")
except Exception as e:
    logger.error(f"Failed to load disaster classification model: {e}")
    disaster_classifier = None

2025-04-04 16:09:12,715 - __main__ - INFO - Loaded spaCy model
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
2025-04-04 16:09:14,154 - __main__ - INFO - Loaded BERT NER model
Device set to use cpu
2025-04-04 16:09:15,981 - __main__ - INFO - Loaded advanced NER model


: 

In [ ]:
def setup_twitter_api():
    """Set up Twitter API connection using tweepy"""
    try:
    
        consumer_key = "Dh7fqMnZ5jwjnNfAWkyHQZ4oZ"
        consumer_secret = "ie5wbRumiTZCrFZaFKulJkIhkYz8ISqWhEjPCorbcBElE66rwY"
        access_token = "1222955679754752000-ovmXhFKfsjE31IETm4Rle2xyF6n78m"
        access_token_secret = "leSB6TlOtppIwgAtFOPGMNsnMZLkKIEbH7SYlYantsGGM"
        bearer_token = "AAAAAAAAAAAAAAAAAAAAAJeJ0QEAAAAA96O7kdxKzyJCN4EJquDBjsutMvo%3D9GIgE3np6a1Y2j0mY4h5sSXfWWSVtWjckuhua9c1zQbi5EX24t"


        
        # Authentication with Twitter
        auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        
        # Test the connection
        api.verify_credentials()
        logger.info("Twitter API connection established successfully")
        return api
    except Exception as e:
        logger.error(f"Failed to connect to Twitter API: {e}")
        return None

def fetch_twitter_data(api, query, hours_ago=6, max_tweets=100):
    """Fetch tweets related to disaster query from the past few hours"""
    if not api:
        logger.error("Twitter API not initialized")
        return []
    
    tweets = []
    try:
        # Calculate time range
        current_time = datetime.utcnow()
        time_ago = current_time - timedelta(hours=hours_ago)
        
        # Format date for Twitter search
        date_since = time_ago.strftime('%Y-%m-%d')
        
        # Search query with disaster term and filter for English tweets
        search_query = f"{query} -filter:retweets lang:en"
        
        # Fetch tweets
        fetched_tweets = tweepy.Cursor(
            api.search_tweets,
            q=search_query,
            lang="en",
            since_id=date_since,
            tweet_mode="extended"
        ).items(max_tweets)
        
        # Process tweets
        for tweet in fetched_tweets:
            # Check if tweet is within time range
            tweet_time = tweet.created_at
            if tweet_time >= time_ago:
                tweet_data = {
                    'id': tweet.id_str,
                    'text': tweet.full_text if hasattr(tweet, 'full_text') else tweet.text,
                    'created_at': tweet_time,
                    'user': tweet.user.screen_name,
                    'location': tweet.user.location,
                    'coordinates': tweet.coordinates,
                    'place': tweet.place.full_name if tweet.place else None,
                    'retweet_count': tweet.retweet_count,
                    'favorite_count': tweet.favorite_count,
                    'query': query
                }
                tweets.append(tweet_data)
        
        logger.info(f"Fetched {len(tweets)} tweets for query: {query}")
    except Exception as e:
        logger.error(f"Error fetching tweets for query {query}: {e}")
    
    return tweets

def process_twitter_data(tweets):
    """Process tweets to extract disaster information"""
    processed_data = []
    
    for tweet in tweets:
        try:
            # Extract text
            text = tweet['text']
            
            # Extract locations
            locations = []
            if tweet['place']:
                locations.append(tweet['place'])
            if tweet['location']:
                locations.append(tweet['location'])
            
            # If coordinates are directly available, use them
            coordinates = None
            if tweet['coordinates'] and tweet['coordinates']['coordinates']:
                # Twitter uses [longitude, latitude] format
                lon, lat = tweet['coordinates']['coordinates']
                coordinates = (lat, lon)
            
            # If no coordinates but locations found, geocode them
            if not coordinates and locations:
                for location in locations:
                    coords = location_to_coordinates(location)
                    if coords:
                        coordinates = coords
                        break
            
            # Extract impact variables
            deaths, injured, affected, homeless = extract_impact_variables(text)
            
            # Calculate severity score
            severity_score = continuous_severity_classification(deaths, injured, affected, homeless)
            
            # Analyze sentiment
            sentiment = analyze_sentiment(text)
            
            # Store the disaster type
            disaster_type = tweet['query']
            
            # Create event data
            if coordinates:
                event_data = {
                    'source': 'twitter',
                    'disaster_type': disaster_type,
                    'location': locations[0] if locations else "Unknown",
                    'latitude': coordinates[0],
                    'longitude': coordinates[1],
                    'date': tweet['created_at'].strftime("%Y-%m-%d"),
                    'time': tweet['created_at'].strftime("%H:%M:%S"),
                    'severity_score': severity_score,
                    'deaths': deaths,
                    'injured': injured,
                    'affected': affected,
                    'homeless': homeless,
                    'sentiment': sentiment['compound'],
                    'url': f"https://twitter.com/{tweet['user']}/status/{tweet['id']}",
                    'text': text,
                    'retweet_count': tweet['retweet_count'],
                    'favorite_count': tweet['favorite_count']
                }
                processed_data.append(event_data)
        except Exception as e:
            logger.error(f"Error processing tweet {tweet.get('id', 'unknown')}: {e}")
    
    return processed_data

def analyze_twitter_trends(twitter_data):
    """Analyze trends in Twitter data related to disasters"""
    if not twitter_data:
        logger.warning("No Twitter data to analyze")
        return None
    
    # Convert to DataFrame if not already
    if not isinstance(twitter_data, pd.DataFrame):
        df = pd.DataFrame(twitter_data)
    else:
        df = twitter_data.copy()
    
    # Calculate engagement metrics
    df['engagement'] = df['retweet_count'] + df['favorite_count']
    
    # Find most engaging tweets
    top_tweets = df.sort_values('engagement', ascending=False).head(10)
    
    # Analyze hashtags
    hashtags = []
    hashtag_pattern = r'#(\w+)'
    for text in df['text']:
        matches = re.findall(hashtag_pattern, text)
        hashtags.extend([tag.lower() for tag in matches])
    
    hashtag_counts = Counter(hashtags)
    top_hashtags = hashtag_counts.most_common(20)
    
    # Analyze mentions
    mentions = []
    mention_pattern = r'@(\w+)'
    for text in df['text']:
        matches = re.findall(mention_pattern, text)
        mentions.extend([mention.lower() for mention in matches])
    
    mention_counts = Counter(mentions)
    top_mentions = mention_counts.most_common(10)
    
    # Create visualizations
    plt.figure(figsize=(12, 10))
    
    # Plot top hashtags
    plt.subplot(2, 1, 1)
    hashtag_df = pd.DataFrame(top_hashtags, columns=['Hashtag', 'Count'])
    sns.barplot(x='Count', y='Hashtag', data=hashtag_df)
    plt.title('Top Hashtags in Disaster Tweets')
    plt.tight_layout()
    
    # Plot engagement by disaster type
    plt.subplot(2, 1, 2)
    engagement_by_type = df.groupby('disaster_type')['engagement'].mean().sort_values(ascending=False)
    engagement_by_type.plot(kind='bar')
    plt.title('Average Engagement by Disaster Type')
    plt.ylabel('Average Engagement (Retweets + Favorites)')
    plt.xlabel('Disaster Type')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Save the visualization
    viz_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "twitter_analysis.png")
    plt.savefig(viz_file)
    plt.close()
    
    logger.info(f"Twitter analysis visualization saved to {viz_file}")
    
    # Create report
    report = "# Twitter Disaster Data Analysis\n\n"
    report += f"Analysis generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    
    report += "## Summary\n\n"
    report += f"Total tweets analyzed: {len(df)}\n"
    report += f"Disaster types mentioned: {df['disaster_type'].nunique()}\n"
    report += f"Average engagement per tweet: {df['engagement'].mean():.2f}\n\n"
    
    report += "## Top Hashtags\n\n"
    for hashtag, count in top_hashtags:
        report += f"- #{hashtag}: {count}\n"
    
    report += "\n## Top Mentions\n\n"
    for mention, count in top_mentions:
        report += f"- @{mention}: {count}\n"
    
    report += "\n## Most Engaging Tweets\n\n"
    for i, (_, tweet) in enumerate(top_tweets.iterrows()):
        report += f"{i+1}. **{tweet['disaster_type']}** (Engagement: {tweet['engagement']})\n"
        report += f"   {tweet['text'][:100]}...\n"
        report += f"   [View Tweet](https://twitter.com/{tweet['user']}/status/{tweet['id']})\n\n"
    
    # Save report
    report_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "twitter_analysis.md")
    try:
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write(report)
        logger.info(f"Twitter analysis report saved to {report_file}")
    except Exception as e:
        logger.error(f"Failed to save Twitter analysis report: {e}")
    
    return {
        'top_hashtags': top_hashtags,
        'top_mentions': top_mentions,
        'top_tweets': top_tweets.to_dict('records')
    }

In [ ]:
@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException, requests.exceptions.HTTPError), max_tries=5)
def fetch_article_content(url):
    """Fetch article content from URL with error handling and backoff"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, timeout=10, headers=headers)  # Set a timeout to avoid hanging
        response.raise_for_status()  # Raise HTTPError for bad responses
        return response.content
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error occurred: {http_err} - {url}")
        raise
    except requests.exceptions.ConnectionError as conn_err:
        logger.error(f"Connection error occurred: {conn_err} - {url}")
        raise
    except requests.exceptions.Timeout as timeout_err:
        logger.error(f"Timeout error occurred: {timeout_err} - {url}")
        raise
    except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error occurred: {req_err} - {url}")
        raise

def parse_article_content(html_content):
    """Parse HTML content to extract article text"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Try to find the main article content
    article = soup.find('article')
    if article:
        paragraphs = article.find_all('p')
    else:
        # Try common content containers
        content_div = soup.find('div', class_=['article-content', 'story-body', 'main-content', 'article-body', 'entry-content'])
        if content_div:
            paragraphs = content_div.find_all('p')
        else:
            # Fallback to all paragraphs
            paragraphs = soup.find_all('p')
    
    article_text = ' '.join([para.text for para in paragraphs])
    
    # Clean the text
    article_text = re.sub(r'\s+', ' ', article_text)  # Remove extra whitespace
    article_text = re.sub(r'[^\x00-\x7F]+', '', article_text)  # Remove non-ASCII characters
    
    return article_text

def extract_ner_entities(text, nlp):
    """Extract named entities from text using NER model"""
    # Break text into chunks if it's too long
    max_length = 512
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    
    all_entities = {}
    for chunk in chunks:
        try:
            ner_results = nlp(chunk)
            for entity in ner_results:
                entity_parts = entity['entity'].split('-')
                if len(entity_parts) > 1:
                    entity_type = entity_parts[1]  # Get the entity type
                    entity_text = entity['word']
                    if entity_type not in all_entities:
                        all_entities[entity_type] = []
                    all_entities[entity_type].append(entity_text)
        except Exception as e:
            logger.error(f"Error in NER processing: {e}")
    
    return all_entities

NameError: name 'backoff' is not defined

In [ ]:
def extract_locations(text):
    """Extract location entities from text using NER models"""
    # Try with advanced NER model first if available
    if advanced_nlp_ner:
        try:
            entities = advanced_nlp_ner(text)
            locations = [entity['word'] for entity in entities if entity['entity_group'] in ['LOC', 'GPE']]
            if locations:
                return locations
        except Exception as e:
            logger.warning(f"Advanced NER failed: {e}. Falling back to spaCy.")
    
    # Fallback to spaCy
    doc = nlp_spacy(text)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    
    # Remove duplicates while preserving order
    seen = set()
    return [loc for loc in locations if not (loc in seen or seen.add(loc))]

@backoff.on_exception(backoff.expo, (GeocoderTimedOut, GeocoderServiceError), max_tries=5)
def location_to_coordinates(location):
    """Convert location name to geographic coordinates"""
    geolocator = Nominatim(user_agent="disaster_event_extraction")
    try:
        location_data = geolocator.geocode(location)
        if location_data:
            return (location_data.latitude, location_data.longitude)
        else:
            logger.warning(f"Location not found: {location}")
            return None
    except GeocoderTimedOut:
        logger.warning(f"Geocoding service timed out for location: {location}")
        raise
    except GeocoderServiceError as e:
        logger.warning(f"Geocoding service error for location {location}: {e}")
        raise
    except Exception as e:
        logger.error(f"Error geocoding location {location}: {e}")
        return None

def extract_dates(text):
    """Extract date entities from text"""
    doc = nlp_spacy(text)
    dates = [ent.text for ent in doc.ents if ent.label_ == 'DATE']
    
    # Try to parse dates to a standard format
    parsed_dates = []
    for date_str in dates:
        try:
            # This is a simple approach - in production, use dateparser library
            date_formats = ["%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%B %d, %Y", "%d %B %Y"]
            for fmt in date_formats:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    parsed_dates.append(date_obj)
                    break
                except ValueError:
                    continue
        except Exception as e:
            logger.debug(f"Failed to parse date '{date_str}': {e}")
            continue
    
    return parsed_dates

In [ ]:
def extract_impact_variables(article_text):
    """Extract impact variables like deaths, injuries from text"""
    # Initialize variables
    deaths = 0
    injured = 0
    affected = 0
    homeless = 0
    
    # First try with NER if available
    if nlp_ner:
        entities = extract_ner_entities(article_text, nlp_ner)
        
        # Try to extract using NER entities
        if 'NUM' in entities:
            for num in entities['NUM']:
                try:
                    num_value = int(num)
                    if re.search(r'(death|dead|killed|fatalities)', article_text.lower()):
                        deaths += num_value
                    elif re.search(r'(injured|wounded|hurt)', article_text.lower()):
                        injured += num_value
                    elif re.search(r'(affected|impacted|hit)', article_text.lower()):
                        affected += num_value
                    elif re.search(r'(homeless|displaced|evacuated)', article_text.lower()):
                        homeless += num_value
                except ValueError:
                    pass
    
    # Use regex patterns as a backup or enhancement
    death_patterns = [
        r'(\d+)\s+(?:people|person|individuals?)?\s+(?:were|was|have been|has been)?\s+killed',
        r'death\s+toll\s+(?:of|is|has reached|reached|climbed to)?\s+(\d+)',
        r'(\d+)\s+deaths',
        r'(\d+)\s+dead'
    ]
    
    injured_patterns = [
        r'(\d+)\s+(?:people|person|individuals?)?\s+(?:were|was|have been|has been)?\s+injured',
        r'(\d+)\s+injured',
        r'(\d+)\s+wounded'
    ]
    
    affected_patterns = [
        r'(\d+)\s+(?:people|person|individuals?)?\s+(?:were|was|have been|has been)?\s+affected',
        r'affecting\s+(\d+)\s+people',
        r'(\d+)\s+affected'
    ]
    
    homeless_patterns = [
        r'(\d+)\s+(?:people|person|individuals?)?\s+(?:were|was|have been|has been)?\s+(?:left)?\s+homeless',
        r'(\d+)\s+homeless',
        r'displaced\s+(\d+)\s+people'
    ]
    
    # Extract numbers using regex patterns
    for pattern in death_patterns:
        matches = re.findall(pattern, article_text.lower())
        for match in matches:
            try:
                deaths += int(match)
            except ValueError:
                pass
    
    for pattern in injured_patterns:
        matches = re.findall(pattern, article_text.lower())
        for match in matches:
            try:
                injured += int(match)
            except ValueError:
                pass
    
    for pattern in affected_patterns:
        matches = re.findall(pattern, article_text.lower())
        for match in matches:
            try:
                affected += int(match)
            except ValueError:
                pass
    
    for pattern in homeless_patterns:
        matches = re.findall(pattern, article_text.lower())
        for match in matches:
            try:
                homeless += int(match)
            except ValueError:
                pass
    
    return deaths, injured, affected, homeless

def continuous_severity_classification(deaths, injured, affected, homeless, infrastructure_damage=0, economic_loss=0):
    """Calculate severity score based on impact variables"""
    # Weights for different factors
    weights = {
        'deaths': 0.35,
        'injured': 0.20,
        'affected': 0.15,
        'homeless': 0.15,
        'infrastructure': 0.10,
        'economic': 0.05
    }
    
    # Apply log transformation to handle wide ranges of values
    severity_score = (
        weights['deaths'] * math.log1p(deaths) +
        weights['injured'] * math.log1p(injured) +
        weights['affected'] * math.log1p(affected) +
        weights['homeless'] * math.log1p(homeless) +
        weights['infrastructure'] * math.log1p(infrastructure_damage) +
        weights['economic'] * math.log1p(economic_loss)
    )
    
    return severity_score

def analyze_sentiment(text):
    """Analyze sentiment of text using VADER"""
    try:
        sia = SentimentIntensityAnalyzer()
        sentiment = sia.polarity_scores(text)
        return sentiment
    except Exception as e:
        logger.error(f"Sentiment analysis failed: {e}")
        return {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}

def classify_disaster_type(text):
    """Classify disaster type using zero-shot classification"""
    if not disaster_classifier:
        return None, 0.0
    
    try:
        result = disaster_classifier(
            text, 
            candidate_labels=disaster_queries,
            multi_label=False
        )
        return result['labels'][0], result['scores'][0]
    except Exception as e:
        logger.error(f"Error in disaster classification: {e}")
        return None, 0.0

In [ ]:
def create_interactive_map(locations, severity_scores, disaster_types):
    """Create an interactive map with disaster locations"""
    # Create a map centered at the first valid location or default to world center
    valid_locations = [(lat, lon) for lat, lon in locations if isinstance(lat, (int, float)) and isinstance(lon, (int, float))]
    if valid_locations:
        map_center = valid_locations[0]
    else:
        map_center = [0, 0]
    
    m = folium.Map(location=map_center, zoom_start=2, tiles="OpenStreetMap")
    
    # Add a marker cluster
    marker_cluster = MarkerCluster().add_to(m)
    
    # Normalize severity scores for color intensity
    if severity_scores:
        max_severity = max(severity_scores)
        min_severity = min(severity_scores)
        severity_range = max_severity - min_severity if max_severity != min_severity else 1
    
    # Add markers for each location
    for i, (lat, lon) in enumerate(locations):
        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)):
            # Calculate color based on severity (red for high severity)
            if severity_scores:
                normalized_severity = (severity_scores[i] - min_severity) / severity_range
                color = f'#{int(255 * normalized_severity):02x}0000'  # Red with intensity based on severity
            else:
                color = 'red'
            
            # Create popup content
            popup_content = f"""
            <b>Disaster Type:</b> {disaster_types[i]}<br>
            <b>Severity Score:</b> {severity_scores[i]:.2f}<br>
            <b>Coordinates:</b> {lat:.4f}, {lon:.4f}
            """
            
            # Add marker
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_content, max_width=300),
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(marker_cluster)
    
    # Add heat map layer
    heat_data = [[lat, lon, score] for (lat, lon), score in zip(valid_locations, severity_scores) 
                if isinstance(lat, (int, float)) and isinstance(lon, (int, float))]
    
    if heat_data:
        HeatMap(heat_data).add_to(m)
    
    # Save map to HTML file
    map_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_map.html")
    m.save(map_file)

    with open(map_file, 'r', encoding='utf-8') as f:
        html_content = f.read()
    display(HTML(html_content))
    
    logger.info(f"Interactive map saved to {map_file}")
    
    return map_file

def save_to_excel(data, filename):
    """Save data to Excel file"""
    try:
        df = pd.DataFrame(data)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(filename)), exist_ok=True)
        
        # Save to Excel
        df.to_excel(filename, index=False)
        logger.info(f"Data successfully saved to {filename}")
        return True
    except Exception as e:
        logger.error(f"Error saving data to Excel: {e}")
        return False

def perform_temporal_analysis(events):
    """Perform temporal analysis of disaster events"""
    if not events:
        logger.warning("No events for temporal analysis")
        return None
    
    # Convert to DataFrame
    df = pd.DataFrame(events)
    
    # Ensure datetime format
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
    
    # Group by date and disaster type
    daily_counts = df.groupby([pd.Grouper(key='datetime', freq='D'), 'disaster_type']).size().unstack(fill_value=0)
    
    # Create temporal visualization
    plt.figure(figsize=(14, 8))
    
    # Plot daily counts by disaster type
    daily_counts.plot(kind='line', marker='o', ax=plt.gca())
    
    plt.title('Temporal Distribution of Disaster Events')
    plt.xlabel('Date')
    plt.ylabel('Number of Events')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Disaster Type')
    
    # Format x-axis to show dates nicely
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    # Save the visualization
    temporal_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "temporal_analysis.png")
    plt.savefig(temporal_file)
    plt.close()
    
    logger.info(f"Temporal analysis visualization saved to {temporal_file}")
    
    return temporal_file

def generate_word_cloud(texts, disaster_type=None):
    """Generate word cloud from texts"""
    if not texts:
        logger.warning("No texts for word cloud generation")
        return None
    
    # Combine all texts
    combined_text = ' '.join(texts)
    
    # Tokenize and remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = word_tokenize(combined_text.lower())
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    
    # Create word cloud
    wordcloud = WordCloud(
        width=800, 
        height=400, 
        background_color='white',
        max_words=200,
        contour_width=3,
        contour_color='steelblue'
    ).generate(' '.join(filtered_words))
    
    # Display the word cloud
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    
    title = f"Word Cloud for {disaster_type} Disaster Texts" if disaster_type else "Word Cloud for Disaster Texts"
    plt.title(title)
    plt.show()
    # Save the word cloud
    filename = f"wordcloud_{disaster_type.lower()}.png" if disaster_type else "wordcloud.png"
    wordcloud_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), filename)
    plt.savefig(wordcloud_file)
    plt.close()
    
    logger.info(f"Word cloud saved to {wordcloud_file}")
    
    return wordcloud_file

def generate_disaster_report(events, output_file=None):
    """Generate a comprehensive report of disaster events"""
    if not events:
        logger.warning("No events to generate report")
        return None
    
    # Convert to DataFrame if not already
    if not isinstance(events, pd.DataFrame):
        events_df = pd.DataFrame(events)
    else:
        events_df = events.copy()
    
    # Group by disaster type
    disaster_summary = events_df.groupby('disaster_type').agg({
        'severity_score': ['mean', 'max', 'count'],
        'deaths': 'sum',
        'injured': 'sum',
        'affected': 'sum',
        'homeless': 'sum'
    }).reset_index()
    
    # Flatten multi-level columns
    disaster_summary.columns = ['_'.join(col).strip('_') for col in disaster_summary.columns.values]
    
    # Sort by count and severity
    disaster_summary = disaster_summary.sort_values(['severity_score_count', 'severity_score_max'], 
                                                   ascending=[False, False])
    
    # Generate report text
    report = "# Disaster Events Report\n\n"
    report += f"Report generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    
    report += "## Summary\n\n"
    report += f"Total events detected: {len(events_df)}\n"
    report += f"Disaster types detected: {len(disaster_summary)}\n"
    report += f"Total deaths reported: {events_df['deaths'].sum()}\n"
    report += f"Total injured reported: {events_df['injured'].sum()}\n\n"
    
    report += "## Disaster Types Breakdown\n\n"
    for _, row in disaster_summary.iterrows():
        report += f"### {row['disaster_type'].title()}\n\n"
        report += f"- Events: {int(row['severity_score_count'])}\n"
        report += f"- Average severity: {row['severity_score_mean']:.2f}\n"
        report += f"- Maximum severity: {row['severity_score_max']:.2f}\n"
        report += f"- Deaths: {int(row['deaths'])}\n"
        report += f"- Injured: {int(row['injured'])}\n"
        report += f"- Affected: {int(row['affected'])}\n"
        report += f"- Homeless: {int(row['homeless'])}\n\n"
    
    report += "## Most Severe Events\n\n"
    most_severe = events_df.sort_values('severity_score', ascending=False).head(10)
    for i, (_, event) in enumerate(most_severe.iterrows()):
        report += f"{i+1}. **{event['disaster_type'].title()} in {event['location']}**\n"
        report += f"   - Severity: {event['severity_score']:.2f}\n"
        report += f"   - Date/Time: {event['date']} {event['time']}\n"
        report += f"   - Impact: {int(event['deaths'])} deaths, {int(event['injured'])} injured\n"
        report += f"   - Coordinates: ({event['latitude']:.4f}, {event['longitude']:.4f})\n\n"
    
    # Save report to file if specified
    if output_file:
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(report)
            logger.info(f"Report saved to {output_file}")
        except Exception as e:
            logger.error(f"Failed to save report: {e}")
    
    return report

def visualize_disaster_distribution(events):
    """Create visualizations for disaster distribution"""
    if not isinstance(events, pd.DataFrame):
        events_df = pd.DataFrame(events)
    else:
        events_df = events.copy()
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Disaster types distribution
    disaster_counts = events_df['disaster_type'].value_counts()
    disaster_counts.plot(kind='bar', ax=axes[0, 0], color='skyblue')
    axes[0, 0].set_title('Distribution of Disaster Types')
    axes[0, 0].set_xlabel('Disaster Type')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Severity distribution
    axes[0, 1].hist(events_df['severity_score'], bins=10, color='salmon')
    axes[0, 1].set_title('Distribution of Severity Scores')
    axes[0, 1].set_xlabel('Severity Score')
    axes[0, 1].set_ylabel('Count')
    
    # 3. Impact variables
    impact_vars = ['deaths', 'injured', 'affected', 'homeless']
    impact_sums = [events_df[var].sum() for var in impact_vars]
    axes[1, 0].bar(impact_vars, impact_sums, color=['darkred', 'orange', 'blue', 'green'])
    axes[1, 0].set_title('Total Impact by Category')
    axes[1, 0].set_xlabel('Impact Category')
    axes[1, 0].set_ylabel('Count')
    
    # 4. Temporal distribution (by day)
    events_df['date_only'] = pd.to_datetime(events_df['date'])
    daily_counts = events_df.groupby('date_only').size()
    daily_counts.plot(kind='line', marker='o', ax=axes[1, 1], color='purple')
    axes[1, 1].set_title('Daily Distribution of Events')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Number of Events')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    # Save the visualization
    viz_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_visualization.png")
    plt.savefig(viz_file)
    plt.close()
    
    logger.info(f"Disaster distribution visualization saved to {viz_file}")
    
    return viz_file

def export_for_dashboard(events, output_file=None):
    """Export event data in a format suitable for dashboard visualization"""
    if not isinstance(events, pd.DataFrame):
        events_df = pd.DataFrame(events)
    else:
        events_df = events.copy()
    
    # Create a simplified structure for dashboard
    dashboard_data = {
        "events": events_df.to_dict('records'),
        "summary": {
            "total_events": len(events_df),
            "total_deaths": int(events_df['deaths'].sum()),
            "total_injured": int(events_df['injured'].sum()),
            "total_affected": int(events_df['affected'].sum()),
            "total_homeless": int(events_df['homeless'].sum()),
            "disaster_types": events_df['disaster_type'].value_counts().to_dict(),
            "avg_severity": float(events_df['severity_score'].mean()),
            "max_severity": float(events_df['severity_score'].max()),
            "min_severity": float(events_df['severity_score'].min()),
        },
        "temporal": {
            "dates": events_df.groupby('date').size().to_dict(),
        },
        "generated_at": datetime.now().isoformat()
    }
    
    # Save to JSON file if specified
    if output_file:
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(dashboard_data, f, indent=2, default=str)
            logger.info(f"Dashboard data exported to {output_file}")
        except Exception as e:
            logger.error(f"Failed to export dashboard data: {e}")
    
    return dashboard_data

In [ ]:
def compare_clustering_algorithms(data):
    """Compare different clustering algorithms and select the best one"""
    # Scale the data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Initialize results dictionary
    results = {}
    
    # K-means
    kmeans_scores = []
    max_clusters = min(10, len(data) - 1)  # Limit max clusters
    
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(scaled_data)
        
        # Calculate metrics
        try:
            silhouette = silhouette_score(scaled_data, labels)
        except:
            silhouette = -1
        
        kmeans_scores.append({
            'k': k,
            'silhouette': silhouette,
            'labels': labels
        })
    
    best_kmeans = max(kmeans_scores, key=lambda x: x['silhouette'])
    results['kmeans'] = best_kmeans
    
    # DBSCAN
    eps_values = [0.3, 0.5, 0.7, 1.0]
    min_samples_values = [3, 5, 10]
    
    dbscan_scores = []
    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(scaled_data)
            
            # Skip if all points are noise
            if len(set(labels)) <= 1:
                continue
            
            # Calculate metrics
            try:
                silhouette = silhouette_score(scaled_data, labels)
            except:
                silhouette = -1
            
            dbscan_scores.append({
                'eps': eps,
                'min_samples': min_samples,
                'silhouette': silhouette,
                'labels': labels
            })
    
    if dbscan_scores:
        best_dbscan = max(dbscan_scores, key=lambda x: x['silhouette'])
        results['dbscan'] = best_dbscan
    
    # Hierarchical clustering
    linkage_methods = ['ward', 'complete', 'average']
    hierarchical_scores = []
    
    for linkage in linkage_methods:
        for k in range(2, max_clusters + 1):
            hierarchical = AgglomerativeClustering(n_clusters=k, linkage=linkage)
            labels = hierarchical.fit_predict(scaled_data)
            
            # Calculate metrics
            try:
                silhouette = silhouette_score(scaled_data, labels)
            except:
                silhouette = -1
            
            hierarchical_scores.append({
                'k': k,
                'linkage': linkage,
                'silhouette': silhouette,
                'labels': labels
            })
    
    best_hierarchical = max(hierarchical_scores, key=lambda x: x['silhouette'])
    results['hierarchical'] = best_hierarchical
    
    # HDBSCAN
    try:
        min_cluster_sizes = [3, 5, 10]
        min_samples_values = [3, 5, 10]
        
        hdbscan_scores = []
        for min_cluster_size in min_cluster_sizes:
            for min_samples in min_samples_values:
                clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
                labels = clusterer.fit_predict(scaled_data)
                
                # Skip if all points are noise
                if len(set(labels)) <= 1:
                    continue
                
                # Calculate metrics
                try:
                    silhouette = silhouette_score(scaled_data, labels)
                except:
                    silhouette = -1
                
                hdbscan_scores.append({
                    'min_cluster_size': min_cluster_size,
                    'min_samples': min_samples,
                    'silhouette': silhouette,
                    'labels': labels
                })
        
        if hdbscan_scores:
            best_hdbscan = max(hdbscan_scores, key=lambda x: x['silhouette'])
            results['hdbscan'] = best_hdbscan
    except Exception as e:
        logger.error(f"HDBSCAN failed: {e}")
    
    # Find best algorithm
    best_algorithm = None
    best_score = -float('inf')
    
    for algorithm, result in results.items():
        if result['silhouette'] > best_score:
            best_score = result['silhouette']
            best_algorithm = algorithm
    
    return best_algorithm, results

In [ ]:
def main():
    """Main function to run the disaster event extraction"""
    # Time range for the last 6 hours
    current_time = datetime.utcnow()
    time_6_hours_ago = current_time - timedelta(hours=6)
    
    logger.info(f"Starting disaster event extraction for events since {time_6_hours_ago}")
    
    # Initialize lists to store data
    all_articles = []
    article_metadata = []
    all_twitter_data = []
    
    # Setup Twitter API
    twitter_api = setup_twitter_api()
    twitter_enabled = True  # Flag to track if Twitter API is working
    
    # Fetch and process articles for each disaster type
    for query in disaster_queries:
        logger.info(f"Processing disaster type: {query}")
        
        # 1. Process Google News
        rss_url = f"https://news.google.com/rss/search?q={query}"
        feed = feedparser.parse(rss_url)
        
        for entry in feed.entries:
            published_time = datetime.fromtimestamp(time.mktime(entry.published_parsed))
            if published_time >= time_6_hours_ago:
                url = entry.link
                try:
                    html_content = fetch_article_content(url)
                    if html_content:
                        article_text = parse_article_content(html_content)
                        all_articles.append(article_text)
                        article_metadata.append({
                            'query': query,
                            'url': url,
                            'published_time': published_time
                        })
                except Exception as e:
                    logger.error(f"Failed to process article: {url} - {e}")
        
        # 2. Process Twitter data only if enabled
        if twitter_api and twitter_enabled:
            try:
                tweets = fetch_twitter_data(twitter_api, query, hours_ago=6)
                processed_tweets = process_twitter_data(tweets)
                all_twitter_data.extend(processed_tweets)
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Error fetching tweets for query {query}: {error_msg}")
                # If we get a 403 Forbidden error, disable Twitter for future queries
                if "403 Forbidden" in error_msg:
                    twitter_enabled = False
                    logger.warning("Twitter API access is forbidden. Disabling Twitter data collection.")
    
    # Check if we have any data to process
    if not all_articles and not all_twitter_data:
        logger.warning("No articles or tweets found for processing")
        return
    
    logger.info(f"Collected {len(all_articles)} articles and {len(all_twitter_data)} tweets")
    
    # Process news articles
    news_event_data = []
    
    for i, article in enumerate(all_articles):
        # Extract locations
        locations = extract_locations(article)
        
        # If no locations found, try to extract from title
        if not locations and 'title' in article_metadata[i]:
            title_locations = extract_locations(article_metadata[i].get('title', ''))
            if title_locations:
                locations = title_locations
        
        # Skip articles with no locations
        if not locations:
            continue
            
        # Extract impact variables using BERT and regex
        deaths, injured, affected, homeless = extract_impact_variables(article)
        
        # Calculate severity score
        severity_score = continuous_severity_classification(deaths, injured, affected, homeless)
        
        # Extract dates
        dates = extract_dates(article)
        event_date = dates[0].strftime("%Y-%m-%d") if dates else article_metadata[i]['published_time'].strftime("%Y-%m-%d")
        event_time = article_metadata[i]['published_time'].strftime("%H:%M:%S")
        
        # Analyze sentiment
        sentiment = analyze_sentiment(article)
        
        # Store the disaster type
        disaster_type = article_metadata[i]['query']
        
        # Get coordinates for each location
        for location in locations:
            coords = location_to_coordinates(location)
            if coords and isinstance(coords, tuple):
                news_event_data.append({
                    'source': 'news',
                    'disaster_type': disaster_type,
                    'location': location,
                    'latitude': coords[0],
                    'longitude': coords[1],
                    'date': event_date,
                    'time': event_time,
                    'severity_score': severity_score,
                    'deaths': deaths,
                    'injured': injured,
                    'affected': affected,
                    'homeless': homeless,
                    'sentiment': sentiment['compound'],
                    'url': article_metadata[i]['url']
                })
    
    # Combine news and Twitter data
    all_event_data = news_event_data + all_twitter_data
    
    if not all_event_data:
        logger.warning("No event data extracted")
        return
    
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(all_event_data)
    
    # Remove duplicates based on location and disaster type
    df = df.drop_duplicates(subset=['location', 'disaster_type'])
    
    # Save all events to Excel
    excel_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_events.xlsx")
    save_to_excel(df.to_dict('records'), excel_file)
    
    # Save Twitter data separately if we have any
    if all_twitter_data:
        twitter_df = pd.DataFrame(all_twitter_data)
        twitter_excel_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "twitter_disaster_events.xlsx")
        save_to_excel(twitter_df.to_dict('records'), twitter_excel_file)
        
        # Analyze Twitter trends
        analyze_twitter_trends(twitter_df)
    
    # Continue with clustering and visualization as before
    if len(df) >= 2:  # Need at least 2 points for clustering
        # Prepare feature matrix for clustering
        X = df[['latitude', 'longitude', 'severity_score']].values
        
        # Compare clustering algorithms
        best_algorithm, clustering_results = compare_clustering_algorithms(X)
        
        logger.info(f"Best clustering algorithm: {best_algorithm}")
        
        # Get labels from best algorithm
        labels = clustering_results[best_algorithm]['labels']
        
        # Add cluster labels to DataFrame
        df['cluster'] = labels
        
        # Find most severe events in each cluster
        severe_events = []
        
        for cluster_id in set(labels):
            if cluster_id != -1:  # Skip noise points
                cluster_df = df[df['cluster'] == cluster_id]
                if not cluster_df.empty:
                    most_severe = cluster_df.loc[cluster_df['severity_score'].idxmax()]
                    severe_events.append(most_severe.to_dict())
        
        # Sort by severity score
        severe_events = sorted(severe_events, key=lambda x: x['severity_score'], reverse=True)
        
        # Save severe events to Excel
        severe_excel_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "severe_disaster_events.xlsx")
        save_to_excel(severe_events, severe_excel_file)
        
        # Create interactive map
        map_file = create_interactive_map(
            [(event['latitude'], event['longitude']) for event in severe_events],
            [event['severity_score'] for event in severe_events],
            [event['disaster_type'] for event in severe_events]
        )
        
        # Display the map in the notebook
        from IPython.display import display, HTML
        try:
            with open(map_file, 'r', encoding='utf-8') as f:
                html_content = f.read()
            display(HTML(html_content))
        except Exception as e:
            logger.error(f"Failed to display map in notebook: {e}")
        
        # Perform temporal analysis
        temporal_data = perform_temporal_analysis(df.to_dict('records'))
        
        # Make sure plots are displayed in the notebook
        plt.figure(figsize=(12, 6))
        if isinstance(temporal_data, pd.Series):
            temporal_data.plot(kind='line', marker='o')
            plt.title('Temporal Distribution of Disaster Events')
            plt.xlabel('Time')
            plt.ylabel('Number of Events')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        
        # Generate word clouds
        generate_word_cloud([article for article in all_articles])
        plt.show()  # Make sure the plot is displayed
        
        # Generate word cloud for tweets if we have any
        if all_twitter_data:
            generate_word_cloud([tweet['text'] for tweet in all_twitter_data], disaster_type="Twitter")
            plt.show()  # Make sure the plot is displayed
        
        # Generate comprehensive report
        report_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_report.md")
        generate_disaster_report(df, report_file)
        
        # Create visualizations
        visualize_disaster_distribution(df)
        plt.show()  # Make sure the plot is displayed
        
        # Export data for dashboard
        dashboard_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "dashboard_data.json")
        export_for_dashboard(df, dashboard_file)
        
        # Print the most severe locations with their coordinates
        print("\nMost severe disaster events in the last 6 hours:")
        for i, event in enumerate(severe_events[:10]):  # Show top 10
            source = event.get('source', 'unknown')
            print(f"{i+1}. [{source.upper()}] {event['disaster_type']} in {event['location']} - Severity: {event['severity_score']:.2f}")
            print(f"   Coordinates: ({event['latitude']}, {event['longitude']})")
            print(f"   Date/Time: {event['date']} {event['time']}")
            print(f"   Impact: {event['deaths']} deaths, {event['injured']} injured")
            print()
    else:
        logger.warning("Not enough data points for clustering")
        
        # Still create map with available data
        if not df.empty:
            map_file = create_interactive_map(
                [(row['latitude'], row['longitude']) for _, row in df.iterrows()],
                [row['severity_score'] for _, row in df.iterrows()],
                [row['disaster_type'] for _, row in df.iterrows()]
            )
            
            # Display the map in the notebook
            from IPython.display import display, HTML
            try:
                with open(map_file, 'r', encoding='utf-8') as f:
                    html_content = f.read()
                display(HTML(html_content))
            except Exception as e:
                logger.error(f"Failed to display map in notebook: {e}")
    
    return df

In [ ]:
# Add these lines at the top of your notebook, after your imports
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
def fetch_google_news_data(query, hours_ago=6, max_articles=20):
    """Fetch disaster-related news from Google News RSS feed"""
    articles = []
    try:
        # Format the RSS URL
        rss_url = f"https://news.google.com/rss/search?q={query}&hl=en-US&gl=US&ceid=US:en"
        
        # Parse the feed
        feed = feedparser.parse(rss_url)
        
        # Calculate time range
        current_time = datetime.utcnow()
        time_ago = current_time - timedelta(hours=hours_ago)
        
        # Process each entry
        for entry in feed.entries[:max_articles]:
            try:
                # Parse published time
                published_time = datetime.fromtimestamp(time.mktime(entry.published_parsed))
                
                # Check if article is within time range
                if published_time >= time_ago:
                    article_data = {
                        'title': entry.title,
                        'link': entry.link,
                        'published': published_time,
                        'source': entry.source.title if hasattr(entry, 'source') else "Google News",
                        'query': query
                    }
                    
                    # Fetch and parse article content
                    html_content = fetch_article_content(entry.link)
                    if html_content:
                        article_text = parse_article_content(html_content)
                        article_data['text'] = article_text
                        articles.append(article_data)
            except Exception as e:
                logger.error(f"Error processing news entry: {e}")
                continue
        
        logger.info(f"Fetched {len(articles)} articles for query: {query}")
    except Exception as e:
        logger.error(f"Error fetching Google News for query {query}: {e}")
    
    return articles

def process_news_articles(articles):
    """Process news articles to extract disaster information"""
    processed_data = []
    
    for article in articles:
        try:
            # Extract text
            text = article['text']
            
            # Extract locations
            locations = extract_locations(text)
            
            # If no locations found, try to extract from title
            if not locations:
                title_locations = extract_locations(article['title'])
                if title_locations:
                    locations = title_locations
            
            # Extract impact variables
            deaths, injured, affected, homeless = extract_impact_variables(text)
            
            # Calculate severity score
            severity_score = continuous_severity_classification(deaths, injured, affected, homeless)
            
            # Extract dates
            dates = extract_dates(text)
            event_date = dates[0].strftime("%Y-%m-%d") if dates else article['published'].strftime("%Y-%m-%d")
            event_time = article['published'].strftime("%H:%M:%S")
            
            # Analyze sentiment
            sentiment = analyze_sentiment(text)
            
            # Store the disaster type
            disaster_type = article['query']
            
            # Verify disaster type using content
            verified_type, confidence = classify_disaster_type(text)
            if verified_type and confidence > 0.7:
                disaster_type = verified_type
            
            # Get coordinates for each location
            for location in locations:
                coords = location_to_coordinates(location)
                if coords and isinstance(coords, tuple):
                    event_data = {
                        'source': 'news',
                        'disaster_type': disaster_type,
                        'location': location,
                        'latitude': coords[0],
                        'longitude': coords[1],
                        'date': event_date,
                        'time': event_time,
                        'severity_score': severity_score,
                        'deaths': deaths,
                        'injured': injured,
                        'affected': affected,
                        'homeless': homeless,
                        'sentiment': sentiment['compound'],
                        'url': article['link'],
                        'title': article['title'],
                        'news_source': article['source']
                    }
                    processed_data.append(event_data)
        except Exception as e:
            logger.error(f"Error processing article {article.get('link', 'unknown')}: {e}")
    
    return processed_data

# Update main function to use enhanced Google News extraction
def main():
    """Main function to run the disaster event extraction"""
    # Time range for the last 6 hours
    current_time = datetime.utcnow()
    time_6_hours_ago = current_time - timedelta(hours=6)
    
    logger.info(f"Starting disaster event extraction for events since {time_6_hours_ago}")
    
    # Initialize lists to store data
    all_news_data = []
    
    # Fetch and process articles for each disaster type
    for query in disaster_queries:
        logger.info(f"Processing disaster type: {query}")
        
        # Process Google News with enhanced extraction
        articles = fetch_google_news_data(query, hours_ago=12, max_articles=30)  # Increased time range and article count
        processed_articles = process_news_articles(articles)
        all_news_data.extend(processed_articles)
    
    if not all_news_data:
        logger.warning("No news articles found for processing")
        return
    
    logger.info(f"Collected {len(all_news_data)} processed news articles")
    
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(all_news_data)
    
    # Remove duplicates based on location and disaster type
    df = df.drop_duplicates(subset=['location', 'disaster_type'])
    
    # Save all events to Excel
    excel_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_events.xlsx")
    save_to_excel(df.to_dict('records'), excel_file)
    
    # Continue with clustering and visualization
    if len(df) >= 2:  # Need at least 2 points for clustering
        # Prepare feature matrix for clustering
        X = df[['latitude', 'longitude', 'severity_score']].values
        
        # Compare clustering algorithms
        best_algorithm, clustering_results = compare_clustering_algorithms(X)
        
        logger.info(f"Best clustering algorithm: {best_algorithm}")
        
        # Get labels from best algorithm
        labels = clustering_results[best_algorithm]['labels']
        
        # Add cluster labels to DataFrame
        df['cluster'] = labels
        
        # Find most severe events in each cluster
        severe_events = []
        
        for cluster_id in set(labels):
            if cluster_id != -1:  # Skip noise points
                cluster_df = df[df['cluster'] == cluster_id]
                if not cluster_df.empty:
                    most_severe = cluster_df.loc[cluster_df['severity_score'].idxmax()]
                    severe_events.append(most_severe.to_dict())
        
        # Sort by severity score
        severe_events = sorted(severe_events, key=lambda x: x['severity_score'], reverse=True)
        
        # Save severe events to Excel
        severe_excel_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "severe_disaster_events.xlsx")
        save_to_excel(severe_events, severe_excel_file)
        
        # Create interactive map
        map_file = create_interactive_map(
            [(event['latitude'], event['longitude']) for event in severe_events],
            [event['severity_score'] for event in severe_events],
            [event['disaster_type'] for event in severe_events]
        )
        
        # Perform temporal analysis
        perform_temporal_analysis(df.to_dict('records'))
        
        # Generate word clouds
        generate_word_cloud([article['text'] for article in articles if 'text' in article])
        
        # Generate comprehensive report
        report_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "disaster_report.md")
        generate_disaster_report(df, report_file)
        
        # Create visualizations
        visualize_disaster_distribution(df)
        plt.ion() 
        # Export data for dashboard
        dashboard_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "dashboard_data.json")
        export_for_dashboard(df, dashboard_file)
        
        # Print the most severe locations with their coordinates
        print("\nMost severe disaster events in the last 12 hours:")
        for i, event in enumerate(severe_events[:10]):  # Show top 10
            print(f"{i+1}. {event['disaster_type']} in {event['location']} - Severity: {event['severity_score']:.2f}")
            print(f"   Coordinates: ({event['latitude']}, {event['longitude']})")
            print(f"   Date/Time: {event['date']} {event['time']}")
            print(f"   Impact: {event['deaths']} deaths, {event['injured']} injured")
            print(f"   Source: {event.get('news_source', 'Unknown')}")
            print(f"   URL: {event['url']}")
            print()
    else:
        logger.warning("Not enough data points for clustering")
        
        # Still create map with available data
        if not df.empty:
            map_file = create_interactive_map(
                [(row['latitude'], row['longitude']) for _, row in df.iterrows()],
                [row['severity_score'] for _, row in df.iterrows()],
                [row['disaster_type'] for _, row in df.iterrows()]
            )
    
    return df

def analyze_news_sources(news_data):
    """Analyze distribution and reliability of news sources"""
    if not news_data:
        logger.warning("No news data to analyze")
        return None
    
    # Convert to DataFrame if not already
    if not isinstance(news_data, pd.DataFrame):
        df = pd.DataFrame(news_data)
    else:
        df = news_data.copy()
    
    # Count articles by source
    source_counts = df['news_source'].value_counts()
    
    # Calculate average severity by source
    severity_by_source = df.groupby('news_source')['severity_score'].mean().sort_values(ascending=False)
    
    # Create visualizations
    plt.figure(figsize=(12, 10))
    
    # Plot sources by article count
    plt.subplot(2, 1, 1)
    source_counts.head(15).plot(kind='bar', color='skyblue')
    plt.title('Top News Sources by Article Count')
    plt.xlabel('News Source')
    plt.ylabel('Number of Articles')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Plot sources by average severity
    plt.subplot(2, 1, 2)
    severity_by_source.head(15).plot(kind='bar', color='salmon')
    plt.title('Top News Sources by Average Severity Score')
    plt.xlabel('News Source')
    plt.ylabel('Average Severity Score')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Display the plot in the notebook
    plt.show()
    
    # Save the visualization
    viz_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "news_source_analysis.png")
    plt.savefig(viz_file)
    plt.close()
    
    logger.info(f"News source analysis visualization saved to {viz_file}")
    
    # Rest of the function remains the same...
    
    # Create report
    report = "# News Source Analysis\n\n"
    report += f"Analysis generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    
    report += "## Summary\n\n"
    report += f"Total news articles analyzed: {len(df)}\n"
    report += f"Number of unique news sources: {df['news_source'].nunique()}\n\n"
    
    report += "## Top News Sources by Article Count\n\n"
    for source, count in source_counts.head(15).items():
        report += f"- {source}: {count} articles\n"
    
    report += "\n## Top News Sources by Average Severity\n\n"
    for source, severity in severity_by_source.head(15).items():
        report += f"- {source}: {severity:.2f} average severity\n"
    
    # Save report
    report_file = os.path.join(os.path.dirname(os.path.abspath(__file__)), "news_source_analysis.md")
    try:
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write(report)
        logger.info(f"News source analysis report saved to {report_file}")
    except Exception as e:
        logger.error(f"Failed to save news source analysis report: {e}")
    
    return {
        'source_counts': source_counts.to_dict(),
        'severity_by_source': severity_by_source.to_dict()
    }